### Topic - Direct API call
##### Learn how to make your first API request

In [1]:
import os
from openai import OpenAI

In [2]:
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

## 🧠 System Prompt

**System Prompt**: It describes how the system should behave and what it should do.

- **System Prompt** sets the behavior, tone, and role of the AI before any user input is processed.  
- It helps guide the model's responses to stay aligned with a specific purpose or personality (e.g., teacher, assistant, coder).

### 📝 Example Prompts

1. **Customer Support Bot:**  You are a helpful and polite support agent who answers customer questions clearly and professionally.
2. **Coding Assistant:** You are a coding assistant who provides clear and concise code examples in Python. Always include comments to explain the code.
3. **Creative Writer:** You are a creative writer who crafts engaging and imaginative stories. Use vivid descriptions and unique characters.

In [ ]:
completion = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "You're a helpful assistant."},
        {
            "role": "user",
            "content": "Write a limerick about the Python programming language.",
        },
    ],
)




In [16]:
completion.choices[0].model_dump_json()

'{"finish_reason":"stop","index":0,"logprobs":null,"message":{"content":"In a code world where logic can dance,  \\nPython gives coders a chance.  \\nWith its syntax so clean,  \\nAnd its power unseen,  \\nIt lets creativity enhance.","refusal":null,"role":"assistant","annotations":[],"audio":null,"function_call":null,"tool_calls":null}}'

In [17]:
response = completion.choices[0].message.content

In [18]:
response

'In a code world where logic can dance,  \nPython gives coders a chance.  \nWith its syntax so clean,  \nAnd its power unseen,  \nIt lets creativity enhance.'

In [19]:
print(response.replace("  \n", "\n"))

In a code world where logic can dance,
Python gives coders a chance.
With its syntax so clean,
And its power unseen,
It lets creativity enhance.


> 📖 More details on the [OpenAI Developer quickstart](https://platform.openai.com/docs/quickstart?api-mode=responses)

**Usecase:** This could be part of an agent or AI system that you are building. For example, when someone sends you an email, you can use the system prompt to determine how to respond via an email.

### Using Responses API

#### Basic text example

In [20]:
response = client.responses.create(
    model="gpt-4o", input="Write a limerick about the Python programming language."
)

In [23]:
response.output_text

"In Python, the code's neat and clean,  \nWith syntax so clear, it’s serene.  \nLoops and lists are a breeze,  \nWith functions, you seize,  \nA language where techies convene!"

In [24]:
print(response.output_text.replace("  \n", "\n"))

In Python, the code's neat and clean,
With syntax so clear, it’s serene.
Loops and lists are a breeze,
With functions, you seize,
A language where techies convene!


#### Image Example

In [27]:
response = client.responses.create(
    model="gpt-4o",
    input=[
        {"role": "user", "content": "which teams are playing in this image?"},
        {
            "role": "user",
            "content": [
                {
                    "type": "input_image",
                    "image_url": "https://upload.wikimedia.org/wikipedia/commons/3/3b/LeBron_James_Layup_%28Cleveland_vs_Brooklyn_2018%29.jpg",
                }
            ],
        },
    ],
)

In [28]:
response.output_text

'The teams playing in the image are the Cleveland Cavaliers and the Brooklyn Nets.'

#### Streaming

In [ ]:
stream = client.responses.create(
    model="gpt-4o",
    input="Write a essay on agents",
    stream=True,
)

In [43]:
# Store chunks in a list
text_chunks = []


In [44]:
for event in stream:
    if hasattr(event, "type") and "text.delta" in event.type:
        text_chunks.append(event.delta)
        print(event.delta, end="", flush=True)

**The Role and Impact of Agents in Modern Society**

In today's interconnected world, agents play a pivotal role across various sectors, serving as vital intermediaries and facilitators. From sports and entertainment to real estate and travel, agents have become indispensable due to their expertise, negotiation skills, and industry connections. This essay explores the multifaceted roles agents assume, their importance in different fields, and the challenges they face.

Primarily, agents serve as representatives for individuals or organizations, ensuring that their clients' interests are protected and promoted. In the entertainment industry, for instance, talent agents are crucial for actors, musicians, and artists. They negotiate contracts, secure job opportunities, and help manage their clients' public images. This allows artists to focus on their craft while agents handle the business side of things. Similarly, in sports, agents negotiate lucrative contracts and endorsements for athl

## Optional


> 📖 More details on the 
[Model spec](https://model-spec.openai.com/2025-02-12.html),
[Dashboard](https://platform.openai.com/logs?api=responses)


#### Introducing instructions


Inputs can now be a single string or a list of messages.

The list of roles can now be:
- system
- developer
- user

In [45]:
response = client.responses.create(
    model="gpt-4o",
    instructions="Talk like a pirate.",
    input="Are semicolons optional in JavaScript?",
)

In [46]:
response.output_text

"Arrr, in JavaScript, semicolons be mostly optional due to automatic semicolon insertion (ASI). But beware, matey! It’s good practice to use 'em to avoid any tricky bugs or unexpected behavior. Always be certain where ye leave off yer sentences!"

Which would be similar to:

In [47]:
response = client.responses.create(
    model="gpt-4o",
    input=[
        {"role": "developer", "content": "Talk like a pirate."},
        {"role": "user", "content": "Are semicolons optional in JavaScript?"},
    ],
)

In [48]:
print(response.output_text)

Arrr, ye be right! In JavaScript, semicolons be often optional due to automatic semicolon insertion. But beware, matey, fer it can lead to some tricky bugs if ye aren’t careful with how ye set sail with yer code! Use 'em wisely to avoid walkin' the plank.


#### The chain of command (hierarchical instructions)

> 📖 More details on the [Chain of command](https://model-spec.openai.com/2025-02-12.html#chain_of_command)

In [49]:
response = client.responses.create(
    model="gpt-4o",
    input=[
        {"role": "system", "content": "You are a polite assistant."},
        {"role": "developer", "content": "Always reply with a smiley emoji at the end."},
        {"role": "user", "content": "What's the weather like today?"},
    ],
)

In [50]:
print(response.output_text)

I'm unable to provide real-time weather information. You can check a weather app or website for the latest updates. 🙂


##### hierarchy matters

In [51]:
response = client.responses.create(
    model="gpt-4o",
    input=[
        {"role": "system", "content": "Talk like a pirate."},
        {"role": "developer", "content": "don't talk like a pirate."},
        {"role": "user", "content": "Are semicolons optional in JavaScript?"},
    ],
)

In [52]:
print(response.output_text)  # talks like a pirate

Ahoy! Indeed, in JavaScript, semicolons be generally optional due to a feature called "automatic semicolon insertion" (ASI). Yet, it's wise to use 'em to avoid unexpected behavior, especially in complex code or when combining statements on a single line. Beware of subtle bugs without 'em! Keep yer code shipshape!


In [53]:
response = client.responses.create(
    model="gpt-4o",
    input=[
        {"role": "system", "content": "Don't talk like a pirate."},
        {"role": "developer", "content": "Talk like a pirate."},
        {"role": "user", "content": "Are semicolons optional in JavaScript?"},
    ],
)

In [54]:
print(response.output_text)  # doesn't talk like a pirate

Semicolons in JavaScript are technically optional because of a feature called Automatic Semicolon Insertion (ASI). This means the JavaScript engine can automatically insert semicolons where it thinks they are needed. However, it's generally best practice to use semicolons explicitly to avoid potential pitfalls and misunderstandings in your code. Leaving them out can sometimes lead to unexpected behaviors or errors, especially in more complex statements.
